***GENERATED CODE FOR datapreparation PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Date ', 'transformation_label': 'String Indexer'}], 'feature': ' Date ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '312', 'mean': '', 'stddev': '', 'min': '1/1/2022', 'max': '9/7/2022', 'missing': '0', 'distinct': '270'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Date '}, {'feature_label': ' Date ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Date ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Country ', 'transformation_label': 'String Indexer'}], 'feature': ' Country ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '312', 'mean': '', 'stddev': '', 'min': ' Australia ', 'max': ' USA ', 'missing': '0', 'distinct': '21'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Country '}, {'feature_label': ' Country ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Country ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' State ', 'transformation_label': 'String Indexer'}], 'feature': ' State ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '312', 'mean': '', 'stddev': '', 'min': ' - ', 'max': ' WY ', 'missing': '0', 'distinct': '35'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' State '}, {'feature_label': ' State ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' State ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Description ', 'transformation_label': 'String Indexer'}], 'feature': ' Description ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '312', 'mean': '', 'stddev': '', 'min': '  Tesla driver hit in multi-car accident  ', 'max': ' [REDACTED] ', 'missing': '0', 'distinct': '274'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Description '}, {'feature_label': ' Description ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Description ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Tesla driver ', 'transformation_label': 'String Indexer'}], 'feature': ' Tesla driver ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '312', 'mean': '1.0', 'stddev': '0.0', 'min': ' - ', 'max': '1', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Tesla driver '}, {'feature_label': ' Tesla driver ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Tesla driver ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Tesla occupant ', 'transformation_label': 'String Indexer'}], 'feature': ' Tesla occupant ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '312', 'mean': '1.29', 'stddev': '0.62', 'min': ' - ', 'max': '3', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Tesla occupant '}, {'feature_label': ' Tesla occupant ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Tesla occupant ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Other vehicle ', 'transformation_label': 'String Indexer'}], 'feature': ' Other vehicle ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '312', 'mean': '1.27', 'stddev': '0.71', 'min': ' - ', 'max': '4', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Other vehicle '}, {'feature_label': ' Other vehicle ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Other vehicle ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Cyclists/ Peds ', 'transformation_label': 'String Indexer'}], 'feature': ' Cyclists/ Peds ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '312', 'mean': '1.11', 'stddev': '0.32', 'min': ' - ', 'max': '2', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Cyclists/ Peds '}, {'feature_label': ' Cyclists/ Peds ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Cyclists/ Peds ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' TSLA+cycl / peds ', 'transformation_label': 'String Indexer'}], 'feature': ' TSLA+cycl / peds ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '312', 'mean': '1.27', 'stddev': '0.64', 'min': ' - ', 'max': '4', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' TSLA+cycl / peds '}, {'feature_label': ' TSLA+cycl / peds ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' TSLA+cycl / peds ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Model ', 'transformation_label': 'String Indexer'}], 'feature': ' Model ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '312', 'mean': '3.0', 'stddev': '0.0', 'min': ' - ', 'max': '3', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Model '}, {'feature_label': ' Model ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Model ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Autopilot claimed ', 'transformation_label': 'String Indexer'}], 'feature': ' Autopilot claimed ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '312', 'mean': '1.18', 'stddev': '0.39', 'min': ' - ', 'max': '2', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Autopilot claimed '}, {'feature_label': ' Autopilot claimed ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Autopilot claimed ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Verified Tesla Autopilot Deaths ', 'transformation_label': 'String Indexer'}], 'feature': ' Verified Tesla Autopilot Deaths ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '312', 'mean': '1.19', 'stddev': '0.11', 'min': '1.0', 'max': '2.0', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Verified Tesla Autopilot...'}, {'feature_label': ' Verified Tesla Autopilot Deaths ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            ' Verified Tesla Autopilot Deaths ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Source', 'transformation_label': 'String Indexer'}], 'feature': 'Source', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '312', 'mean': '', 'stddev': '', 'min': ' NHTSA SGO ', 'max': ' https://www.uppermichiganssource.com/2021/06/19/2-dead-after-1-car-crash-marquette-county-road-550/ ', 'missing': '0', 'distinct': '301'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Source'}, {'feature_label': 'Source', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Source')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Note ', 'transformation_label': 'String Indexer'}], 'feature': ' Note ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '312', 'mean': '', 'stddev': '', 'min': ' AP claimed in NHTSA complaint ', 'max': "Names of deceased came from a relative's email", 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Note '}, {'feature_label': ' Note ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Note ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Deceased 1 ', 'transformation_label': 'String Indexer'}], 'feature': ' Deceased 1 ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '312', 'mean': '', 'stddev': '', 'min': ' Alfred Valles ', 'max': ' Zhang ', 'missing': '0', 'distinct': '101'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Deceased 1 '}, {'feature_label': ' Deceased 1 ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Deceased 1 ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Deceased 2 ', 'transformation_label': 'String Indexer'}], 'feature': ' Deceased 2 ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '312', 'mean': '', 'stddev': '', 'min': ' Alex Francisco Cuesta ', 'max': ' William Price ', 'missing': '0', 'distinct': '26'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Deceased 2 '}, {'feature_label': ' Deceased 2 ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Deceased 2 ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Deceased 3 ', 'transformation_label': 'String Indexer'}], 'feature': ' Deceased 3 ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '312', 'mean': '', 'stddev': '', 'min': ' Andrew James Chaves ', 'max': ' Quinn Cochlin ', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Deceased 3 '}, {'feature_label': ' Deceased 3 ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Deceased 3 ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Deceased 4 ', 'transformation_label': 'String Indexer'}], 'feature': ' Deceased 4 ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '312', 'mean': '', 'stddev': '', 'min': ' Patricia A. O’Neill ', 'max': ' Patricia A. O’Neill ', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Deceased 4 '}, {'feature_label': ' Deceased 4 ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Deceased 4 ')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run datapreparationHooks.ipynb
try:
	#sourcePreExecutionHook()

	tesladeathsdatasetn = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Tesla Deaths - Dataset_N.csv', 'filename': 'Tesla Deaths - Dataset_N.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run datapreparationHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofenewdataset = TransformationMain.run(tesladeathsdatasetn,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": " Case # ", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "312", "mean": "234.74", "stddev": "90.57", "min": "79", "max": "392", "missing": "0"}, "updatedLabel": " Case # "}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": " Year ", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "312", "mean": "2021.66", "stddev": "1.28", "min": "2019", "max": "2024", "missing": "0"}, "updatedLabel": " Year "}, {"transformationsData": [{"feature_label": " Date ", "transformation_label": "String Indexer"}], "feature": " Date ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "312", "mean": "", "stddev": "", "min": "1/1/2022", "max": "9/7/2022", "missing": "0", "distinct": "270"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Date "}, {"transformationsData": [{"feature_label": " Country ", "transformation_label": "String Indexer"}], "feature": " Country ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "312", "mean": "", "stddev": "", "min": " Australia ", "max": " USA ", "missing": "0", "distinct": "21"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Country "}, {"transformationsData": [{"feature_label": " State ", "transformation_label": "String Indexer"}], "feature": " State ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "312", "mean": "", "stddev": "", "min": " - ", "max": " WY ", "missing": "0", "distinct": "35"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " State "}, {"transformationsData": [{"feature_label": " Description ", "transformation_label": "String Indexer"}], "feature": " Description ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "312", "mean": "", "stddev": "", "min": "  Tesla driver hit in multi-car accident  ", "max": " [REDACTED] ", "missing": "0", "distinct": "274"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Description "}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": " Deaths ", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "312", "mean": "1.28", "stddev": "0.62", "min": "1", "max": "4", "missing": "0"}, "updatedLabel": " Deaths "}, {"transformationsData": [{"feature_label": " Tesla driver ", "transformation_label": "String Indexer"}], "feature": " Tesla driver ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "312", "mean": "1.0", "stddev": "0.0", "min": " - ", "max": "1", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Tesla driver "}, {"transformationsData": [{"feature_label": " Tesla occupant ", "transformation_label": "String Indexer"}], "feature": " Tesla occupant ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "312", "mean": "1.29", "stddev": "0.62", "min": " - ", "max": "3", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Tesla occupant "}, {"transformationsData": [{"feature_label": " Other vehicle ", "transformation_label": "String Indexer"}], "feature": " Other vehicle ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "312", "mean": "1.27", "stddev": "0.71", "min": " - ", "max": "4", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Other vehicle "}, {"transformationsData": [{"feature_label": " Cyclists/ Peds ", "transformation_label": "String Indexer"}], "feature": " Cyclists/ Peds ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "312", "mean": "1.11", "stddev": "0.32", "min": " - ", "max": "2", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Cyclists/ Peds "}, {"transformationsData": [{"feature_label": " TSLA+cycl / peds ", "transformation_label": "String Indexer"}], "feature": " TSLA+cycl / peds ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "312", "mean": "1.27", "stddev": "0.64", "min": " - ", "max": "4", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " TSLA+cycl / peds "}, {"transformationsData": [{"feature_label": " Model ", "transformation_label": "String Indexer"}], "feature": " Model ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "312", "mean": "3.0", "stddev": "0.0", "min": " - ", "max": "3", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Model "}, {"transformationsData": [{"feature_label": " Autopilot claimed ", "transformation_label": "String Indexer"}], "feature": " Autopilot claimed ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "312", "mean": "1.18", "stddev": "0.39", "min": " - ", "max": "2", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Autopilot claimed "}, {"transformationsData": [{"feature_label": " Verified Tesla Autopilot Deaths ", "transformation_label": "String Indexer"}], "feature": " Verified Tesla Autopilot Deaths ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "312", "mean": "1.19", "stddev": "0.11", "min": "1.0", "max": "2.0", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Verified Tesla Autopilot..."}, {"transformationsData": [{"feature_label": "Source", "transformation_label": "String Indexer"}], "feature": "Source", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "312", "mean": "", "stddev": "", "min": " NHTSA SGO ", "max": " https://www.uppermichiganssource.com/2021/06/19/2-dead-after-1-car-crash-marquette-county-road-550/ ", "missing": "0", "distinct": "301"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Source"}, {"transformationsData": [{"feature_label": " Note ", "transformation_label": "String Indexer"}], "feature": " Note ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "312", "mean": "", "stddev": "", "min": " AP claimed in NHTSA complaint ", "max": "Names of deceased came from a relative's email", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Note "}, {"transformationsData": [{"feature_label": " Deceased 1 ", "transformation_label": "String Indexer"}], "feature": " Deceased 1 ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "312", "mean": "", "stddev": "", "min": " Alfred Valles ", "max": " Zhang ", "missing": "0", "distinct": "101"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Deceased 1 "}, {"transformationsData": [{"feature_label": " Deceased 2 ", "transformation_label": "String Indexer"}], "feature": " Deceased 2 ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "312", "mean": "", "stddev": "", "min": " Alex Francisco Cuesta ", "max": " William Price ", "missing": "0", "distinct": "26"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Deceased 2 "}, {"transformationsData": [{"feature_label": " Deceased 3 ", "transformation_label": "String Indexer"}], "feature": " Deceased 3 ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "312", "mean": "", "stddev": "", "min": " Andrew James Chaves ", "max": " Quinn Cochlin ", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Deceased 3 "}, {"transformationsData": [{"feature_label": " Deceased 4 ", "transformation_label": "String Indexer"}], "feature": " Deceased 4 ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "312", "mean": "", "stddev": "", "min": " Patricia A. O\u2019Neill ", "max": " Patricia A. O\u2019Neill ", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Deceased 4 "}]}))

	#transformationPostExecutionHook(autofenewdataset)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run datapreparationHooks.ipynb
try:
	#sinkPreExecutionHook()

	teslamodelsaccidentsimpactpredictionoutput = NumtraConnector.fetch(spark, "{'samplefile': {'status': False}, 'samplecount': 390, 'originalcount': 390, 'orignalKey': None, 'pathOnly': '/Tesla Models Accidents Impact Prediction', 'project_id': '65c2eb84ef8cf2dbe1b4e12b', 'parent_id': '65c2eb84ef8cf2dbe1b4e12b', 'original_schema': [], 'actual_schema': [], 'server': 'https://ml.colaberry.com:443', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'CleanedautoFEData.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-4444726e-45bb-4194-82cf-85f6639c0eed-c000.csv', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
